In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers>=4.24.0
%pip install sentencepiece
%pip install accelerate>=0.26.0

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
     --- ------------------------------------ 0.5/5.3 MB 3.3 MB/s eta 0:00:02
     ----------- ---------------------------- 1.6/5.3 MB 6.0 MB/s eta 0:00:01
     ----------------- ---------------------- 2.4/5.3 MB 4.1 MB/s eta 0:00:01
     ----------------------------- ---------- 3.9/5.3 MB 5.7 MB/s eta 0:00:01
     ---------------------------------------- 5.3/5.3 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ---------- ----------------------------- 1.0/4.0 MB 6.3 MB/s eta 0:00:01
     ----------------------- ---------------- 2.4/4.0 MB 5.8 MB/s eta 0:00:01
     --------------------------------- ------ 3.4/4.0 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------- 4.0/4.0 MB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------

In [2]:
%pip install torch pandas numpy matplotlib scikit-learn gradio


   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/11.6 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.6 MB 4.3 MB/s eta 0:00:03
   -------------- ------------------------- 4.2/11.6 MB 6.0 MB/s eta 0:00:02
   ---------------------- ----------------- 6.6/11.6 MB 6.8 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.6 MB 7.9 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.6 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.0 MB 6.7 MB/s eta 0:00:01
   ------------------- -------------------- 3.9/8.0 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.0 MB 9.5 MB/s eta 0:00:01
   -----------

In [3]:
%pip install --upgrade typing_extensions

In [4]:
%pip install textstat

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ------------------------- -------------- 1.3/2.1 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from transformers import (
    DebertaV2Tokenizer,
    DebertaV2ForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
import gradio as gr

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from textstat import flesch_kincaid_grade

# Очистка памяти
gc.collect()
torch.cuda.empty_cache()

class AITextAnalyzer:
    def __init__(self, dataframe, text_column="text"):
        self.formal_words = [
            "вышеупомянутый",
            "надлежит",
            "следует отметить",
            "в заключение",
            "соответственно",
            "следовательно",
            "таким образом",
            "в частности",
            "например",
            "касательно",
            "относительно",
        ]  
        self.introductory_words = [
            "во-первых",
            "во-вторых",
            "в-третьих",
            "следовательно",
            "таким образом",
            "наконец",
            "с другой стороны",
            "итак",
            "впрочем",
        ]  
        self.cliches = [
            "играет важную роль",
            "не секрет, что",
            "стоит отметить, что",
            "в настоящее время",
            "следует подчеркнуть",
        ]  
        self.stop_words = set(stopwords.words("russian"))
        self.human_corpus = self.extract_human_corpus(dataframe, text_column)
        self.human_word_freq = self.calculate_word_frequencies(self.human_corpus)

    def extract_human_corpus(self, dataframe, text_column):

        human_texts = []
        for text in dataframe[text_column]:
            try:
                text_content, label = text.rsplit(",", 1)
                label = int(label.strip())
                if label == 0:
                    human_texts.append(text_content)
            except ValueError:
                print(f"Ошибка в формате строки: {text}. Строка пропущена.")
        if not human_texts:
            print(
                "Предупреждение: Не найдено человеческих текстов (label=0) в DataFrame. "
                "Частотность слов не может быть рассчитана."
            )
            return ""
        return " ".join(human_texts)

    def calculate_word_frequencies(self, text):
        """Рассчитывает частоту каждого слова в корпусе."""
        words = [
            word.lower()
            for word in word_tokenize(text)
            if word.isalpha() and word.lower() not in self.stop_words
        ]
        word_counts = Counter(words)
        total_words = len(words)
        word_frequencies = {word: count / total_words for word, count in word_counts.items()}
        return word_frequencies

    def analyze_text(self, text, bert_probability):
        """
        Анализирует текст на наличие признаков ИИ.

        Args:
            text: Текст для анализа.
            bert_probability: Вероятность того, что текст сгенерирован ИИ, полученная от модели BERT.

        Returns:
            Словарь с результатами анализа.
        """

        analysis_results = {}

        # 1. Стилистический анализ
        analysis_results["style"] = self.analyze_style(text)

        # 2. Статистический анализ
        analysis_results["statistics"] = self.analyze_statistics(text)

        # 3. Объединение результатов и формирование комментария
        analysis_results["comment"] = self.generate_comment(analysis_results, bert_probability)

        return analysis_results

    def analyze_style(self, text):
      
        """Анализирует стилистические особенности текста."""
        style_results = {}

        # 1.1 Слишком формальный или книжный язык
        style_results["formal_language"] = self.detect_formal_language(text)

        # 1.6 Повторы слов и фраз
        style_results["repetitions"] = self.detect_repetitions(text)

        # 1.7 Чрезмерное использование вводных слов и клише
        style_results["introductory_words_and_cliches"] = self.detect_introductory_words_and_cliches(text)

        # 1.8 Редкие/неуместные слова
        style_results["rare_words"] = self.detect_rare_words(text)

        return style_results
    
    def analyze_statistics(self, text):
      """Анализирует статистические параметры текста."""
      statistics_results = {}
      statistics_results["word_frequencies"] = self.analyze_word_frequencies(text)
      statistics_results["sentence_length"] = self.analyze_sentence_length(text)
      statistics_results["readability"] = self.analyze_readability(text)
      statistics_results["entropy"] = self.calculate_entropy(text)
      return statistics_results

    def detect_formal_language(self, text):
      """Выявляет формальные слова и выражения."""
      formal_words_found = [word for word in self.formal_words if word in text.lower()]
      return {
          "score": len(formal_words_found),
          "examples": formal_words_found
      }
    
    def detect_repetitions(self, text):
      """Выявляет повторы слов и фраз."""
      words = [word.lower() for word in word_tokenize(text) if word.isalpha()]
      word_counts = Counter(words)
      repeated_words = {word: count for word, count in word_counts.items() if count > 3 and word not in self.stop_words}

      phrases = [phrase.lower() for phrase in sent_tokenize(text) if len(phrase.split())>1] # Только фразы, состоящие из нескольких слов
      phrase_counts = Counter(phrases)
      repeated_phrases = {phrase: count for phrase, count in phrase_counts.items() if count > 1}

      return {
          "score": sum(repeated_words.values()) + sum(repeated_phrases.values()),
          "words": repeated_words,
          "phrases": repeated_phrases
      }

    def detect_introductory_words_and_cliches(self, text):
      introductory_words_found = [word for word in self.introductory_words if word in text.lower()]
      cliches_found = [cliche for cliche in self.cliches if cliche in text.lower()]
      return {
          "score": len(introductory_words_found) + len(cliches_found),
          "introductory_words": introductory_words_found,
          "cliches": cliches_found
      }

    def detect_rare_words(self, text):
      """Выявляет редкие слова, сравнивая с частотным словарем корпуса, написанного людьми"""
      words = [word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in self.stop_words]
      rare_words = {word: self.human_word_freq.get(word, 0) for word in words}
      rare_words_found = {word: freq for word, freq in rare_words.items() if freq < 0.0001} # Порог редкости
      return {
          "score": len(rare_words_found),
          "examples": rare_words_found
      }

    def analyze_word_frequencies(self, text):
      """Сравнивает частотность слов в тексте с частотностью в корпусе текстов, написанных людьми."""
      words = [word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in self.stop_words]
      word_counts = Counter(words)
      total_words = len(words)
      word_frequencies = {word: count / total_words for word, count in word_counts.items()}

      frequency_differences = {}
      for word, freq in word_frequencies.items():
          human_freq = self.human_word_freq.get(word, 0)
          frequency_differences[word] = freq - human_freq

      # Сортируем по модулю разницы
      sorted_frequency_differences = dict(sorted(frequency_differences.items(), key=lambda item: abs(item[1]), reverse=True))

      return {
          "score": sum(abs(diff) for diff in sorted_frequency_differences.values()),
          "differences": sorted_frequency_differences
      }
    
    def analyze_sentence_length(self, text):
      """Анализирует среднюю длину предложений."""
      sentences = sent_tokenize(text)
      sentence_lengths = [len(word_tokenize(sentence)) for sentence in sentences]
      avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
      return {
          "avg_length": avg_sentence_length,
          "lengths": sentence_lengths,
          "deviation": avg_sentence_length - 15.5 # 15 - средняя длина предложения (примерное значение, нужно уточнить)
      }
    
    def analyze_readability(self, text):
      """Анализирует метрики читабельности."""
      readability_score = flesch_kincaid_grade(text)
      return {
          "flesch_kincaid_grade": readability_score,
          "deviation": readability_score - 10 # 10 - примерный целевой уровень, нужно уточнить
      }

    def calculate_entropy(self, text):
      """Рассчитывает энтропию текста."""
      words = [word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in self.stop_words]
      total_words = len(words)
      if total_words==0:
        return {
            "entropy": 0
        }
      else:
        word_counts = Counter(words)
        probabilities = [count / total_words for count in word_counts.values()]
        entropy = -sum(p * np.log2(p) for p in probabilities)
        return {
            "entropy": entropy
        }

    # --- Метод для генерации комментария ---

    def generate_comment(self, analysis_results, bert_probability):
        """Генерирует объяснительный комментарий на основе результатов анализа."""
        comments = []
        comments.append(f"Вероятность ИИ-текста по модели BERT: {bert_probability:.2f}%")

        # Стилистический анализ
        if analysis_results["style"]["formal_language"]["score"] > 5:  # Порог увеличен
            comments.append("В тексте используется формальный стиль речи, что может указывать на ИИ-генерацию. Примеры: " + ", ".join(
                analysis_results["style"]["formal_language"]["examples"][:3]))

        if analysis_results["style"]["repetitions"]["score"] > 6:  # Порог увеличен
            comments.append(
                "В тексте много повторов слов и фраз, что нехарактерно для человеческой речи. Например, слово '" +
                list(analysis_results["style"]["repetitions"]["words"].keys())[0] + "' повторяется " + str(
                    analysis_results["style"]["repetitions"]["words"][
                        list(analysis_results["style"]["repetitions"]["words"].keys())[0]]) + " раз.")

        if analysis_results["style"]["introductory_words_and_cliches"]["score"] > 3:  # Порог увеличен
            comments.append(
                "Текст содержит много вводных слов и клише, что может свидетельствовать о его искусственном происхождении.")

        # Больше нет комментариев по статистическому анализу!
        if len(comments) > 1:
          return " ".join(comments)
        else:
            return ""

In [17]:
data = pd.read_csv('Data_set_1_0.csv')
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data = data.dropna()
print(f'Количество примеров: {len(data)}')
data.head()

Количество примеров: 19161


,text,label
0,"В лабиринте слов, где каждый звук – отдельный ...",1
1,"В бесконечном океане человеческой мысли, где п...",1
2,курсов. Эту ссуду он должен возвратить во что ...,0
3,чтоб она туда не ходила… на тела… на тела… в к...,0
4,"Яркость и глубина русской речи, её способность...",1


In [18]:
train_df, val_df = train_test_split(data, test_size=0.1, random_state=42)
print(f'Обучающих примеров: {len(train_df)}, Валидационных примеров: {len(val_df)}')

Обучающих примеров: 17244, Валидационных примеров: 1917


In [19]:
MODEL_NAME = 'microsoft/deberta-v3-large'
MAX_LEN = 256
BATCH_SIZE = 32
EPOCHS = 5
OUTPUT_DIR = "debert-large-3_19"

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)

In [20]:

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=MAX_LEN):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = TextDataset(train_df['text'], train_df['label'], tokenizer)
eval_dataset = TextDataset(val_df['text'], val_df['label'], tokenizer)

In [21]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir=OUTPUT_DIR,
    logging_steps=500,
    save_total_limit=2,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    fp16=True,
    optim="adamw_torch"
)

model = DebertaV2ForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=2, 
    force_download=True,
    ignore_mismatched_sizes=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\matia\AppData\Local\Temp\ipykernel_21192\2041116628.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
train_result = trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
metrics = trainer.evaluate()
print("Метрики модели:", metrics)


# Логи обучения
logs = trainer.state.log_history
print(logs)

  0%|          | 0/675 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 1/675 [00:04<54:24,  4.84s/it]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 2/675 [00:06<35:20,  3.15s/it]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 3/675 [00:08<28:56,  2.58s/i

{'loss': 0.1067, 'grad_norm': 0.016728831455111504, 'learning_rate': 2e-05, 'epoch': 3.7}


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

 13%|█▎        | 2/15 [00:01<00:06,  1.91it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

 20%|██        | 3/15 [00:02<00:08,  1.33it/s]/usr/local/lib/python3.10/dist-packages/torch

{'eval_loss': 0.0007258612313307822, 'eval_runtime': 15.9734, 'eval_samples_per_second': 120.012, 'eval_steps_per_second': 0.939, 'epoch': 3.7}


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 74%|███████▍  | 501/675 [16:28<19:34,  6.75s/it]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 74%|███████▍  | 502/675 [16:30<15:22,  5.33s/it]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 75%|███████▍  | 503/675 [16:31<12:20,  4.31s/it]/usr/local/lib/python3.10/dist-

{'train_runtime': 1360.6137, 'train_samples_per_second': 63.368, 'train_steps_per_second': 0.496, 'train_loss': 0.07903858326099537, 'epoch': 5.0}



/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 13%|█▎        | 2/15 [00:01<00:06,  1.91it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 20%|██        | 3/15 [00:02<00:08,  1.34it/s]/usr/local/lib/python3.10/dist-packages/torch/n

Метрики модели: {'eval_loss': 0.0005603526369668543, 'eval_runtime': 15.9452, 'eval_samples_per_second': 120.224, 'eval_steps_per_second': 0.941, 'epoch': 5.0}
[{'loss': 0.1067, 'grad_norm': 0.016728831455111504, 'learning_rate': 2e-05, 'epoch': 3.7037037037037037, 'step': 500}, {'eval_loss': 0.0007258612313307822, 'eval_runtime': 15.9734, 'eval_samples_per_second': 120.012, 'eval_steps_per_second': 0.939, 'epoch': 3.7037037037037037, 'step': 500}, {'train_runtime': 1360.6137, 'train_samples_per_second': 63.368, 'train_steps_per_second': 0.496, 'total_flos': 4.017583229773824e+16, 'train_loss': 0.07903858326099537, 'epoch': 5.0, 'step': 675}, {'eval_loss': 0.0005603526369668543, 'eval_runtime': 15.9452, 'eval_samples_per_second': 120.224, 'eval_steps_per_second': 0.941, 'epoch': 5.0, 'step': 675}]


In [23]:
best_model_path = os.path.join(OUTPUT_DIR, 'checkpoint-best')
if not os.path.exists(best_model_path):
    best_model_path = OUTPUT_DIR

best_model = DebertaV2ForSequenceClassification.from_pretrained(best_model_path)
best_model = best_model.to(device)
best_tokenizer = DebertaV2Tokenizer.from_pretrained(best_model_path)

In [26]:
def predict_and_explain(text):
    analyzer = AITextAnalyzer(data) # Инициализируем анализатор текста
    encoding = best_tokenizer(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        truncation=True,
        padding='max_length',
        return_tensors='pt',
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    best_model.eval()
    with torch.no_grad():
        outputs = best_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        prediction = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][prediction].item() * 100
    label = 'ИИ' if prediction == 1 else 'Человек'

    # Анализируем текст, чтобы получить статистику
    analysis_results = analyzer.analyze_text(text, confidence)
    stats = analysis_results["statistics"]

    # Формируем строку со статистикой
    stats_text = (
        f"Средняя длина предложений: {stats['sentence_length']['avg_length']:.1f} слов\n"
        f"Показатель удобочитаемости Флеша-Кинкейда: {stats['readability']['flesch_kincaid_grade']}\n"
        f"Энтропия: {stats['entropy']['entropy']:.2f}"
    )

    # Формируем итоговый вывод
    # Для ИИ-текста добавляем статистику
    if prediction == 1:
        result = f"{label} (уверенность: {confidence:.2f}%)\n\nСтатистика:\n{stats_text}"
    else:
        result = f"{label} (уверенность: {confidence:.2f}%)"

    return result

title = "Детектор ИИ для русского языка"
iface = gr.Interface(
    fn=predict_and_explain,
    inputs=gr.Textbox(label="Введите текст", lines=5, placeholder="Введите текст здесь..."),
    outputs=gr.Textbox(label="Результат"),
    title=title,
    theme=gr.themes.Default(),
    allow_flagging="never"
)
iface.launch(share=True)

c:\Users\matia\OneDrive\Рабочий стол\Проектик\.venv\lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://0a0ed6245dd0e5823b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "c:\Users\matia\OneDrive\Рабочий стол\Проектик\.venv\lib\site-packages\nltk\corpus\util.py", line 84, in __load
    root = nltk.data.find(f"{self.subdir}/{zip_name}")
  File "c:\Users\matia\OneDrive\Рабочий стол\Проектик\.venv\lib\site-packages\nltk\data.py", line 579, in find
    raise LookupError(resource_not_found)
LookupError: 
**********************************************************************
  Resource stopwords not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('stopwords')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/stopwords.zip/stopwords/

  Searched in:
    - 'C:\\Users\\matia/nltk_data'
    - 'c:\\Users\\matia\\OneDrive\\Рабочий стол\\Проектик\\.venv\\nltk_data'
    - 'c:\\Users\\matia\\OneDrive\\Рабочий стол\\Проектик\\.venv\\share\\nltk_data'
    - 'c:\\Users\\matia\\OneDrive\\Рабочий стол\\Проектик\\.venv\\lib\\nltk_data'
  